# HCI experiment 
Use this jupyter to generate the image stimuli matrix

Goal :
    

Information:
- EEG sampling = 125 Hz with 16 electrod nodes
    
    
Requirements:

1. Genterate represent number 0-9 with 500 number
2. Divide it in to 25 blcok, 20 images per block
3. Current trial should not same number with previous


In [1]:
import random
import pylsl
import numpy as np
import pandas as pd
import time
import itertools
import math
import psychopy 
from psychopy import visual, core, event
from datetime import datetime
from IPython.display import clear_output
import random


In [2]:
#==============================================
# experiment parameters
#==============================================
combination         = 6
task                = 2
classes             = combination * task
num_trial_per_block = 5
num_block           = 15
num_trial           = num_trial_per_block * num_block
total_data_point    = classes * num_trial 

stim_time           = (1, 2)
stim_imagery        = 2
stim_blink_time     = 0
task_flixation_time = (0.8, 1.2)
trial_flixation_time= 2
block_fixation_time = 10  # inter trial interval, i.e., how long the fixation will stay in second


experiment_time  = ( trial_flixation_time + stim_time[-1] + stim_imagery + task_flixation_time[-1] ) * combination * num_trial
print(f"Total experiment time = {'{:.2f}'.format(experiment_time/60)} Minute" )
      
    
#==============================================
# Configuration 
#==============================================
image_folder='./stimulus/large'


#name, type, channel_count, sampling rate, channel format, source_id
#info = StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'int32', 'CytonMarkerID')#make an outlet
info = pylsl.StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'string', 'CytonMarkerID')#make an outlet
outlet = pylsl.StreamOutlet(info)
# %whos

Total experiment time = 54.00 Minute


In [3]:
stims = {
    '1': [1, 2, 3, 4, 5, 6],
    '2': [4, 1, 5, 2 ,6 ,3],
    '3': [5, 3, 1, 6, 4, 2],
    '4': [2, 4, 6, 1, 3, 5],
    '5': [3, 6, 2, 5, 1, 4],
    '6': [6, 5, 4 ,3, 2, 1],
}

## Experiment section

In [4]:
def drawTextOnScreen(massage) :
    message = visual.TextStim( mywin, text=massage, languageStyle='LTR')
    message.contrast =  0.3
    message.height= 0.07
    message.draw() # draw on screen
    mywin.flip()   # refresh to show what we have draw

def drawTrial( fileName, stimTime ) :
    drawTextOnScreen('') 
    core.wait(stim_blink_time)
    if fileName == 'imagery':
        img = blank
    else:
        img = all_img[fileName-1]
    img.draw()
    mywin.flip()
    eegMarking(fileName, "img_stim" )
    core.wait(stimTime)
    
def drawFixation(fileName, fixationTime):
    fixation = visual.ShapeStim(mywin,
                                vertices=((0, -0.5), (0, 0.5), (0,0), (-0.5,0), (0.5, 0)),
                                lineWidth=5,
                                closeShape=False,
                                lineColor="white"
            )
    fixation.draw()
    if not(isTrianing) and fileName == 'trial break':
        text = f"Block {block+1} / {num_block}"
        message = visual.TextStim( mywin, text=text, languageStyle='LTR' )
        message.contrast =  0.3
        message.pos = (0, -0.6)
        message.draw() # draw on screen
        
    mywin.flip()   # refresh to show what we have draw
    eegMarking(fileName, "fixation" )
    core.wait(fixationTime)
    drawTextOnScreen('')
     
def eegMarking(img, stampType):   # use trial variable from main
    if not isTrianing :
        if stampType == "img_stim" :
            markerString = str(block+1) + "," + str(trial) + ","  + str(img) + "," + str(timer)
        elif stampType == "fixation" :
            markerString = str(block+1) + "," +str(img) + "," + str("Fixation")
    else:
        markerString = 'Training'
    markerString= str(markerString)                              
    print("Marker string {}".format(markerString))
    outlet.push_sample([markerString])

### Stimulies

In [5]:
trial_idx = input()

 4


In [6]:
wait = 0.5
    
mywin = visual.Window([1366, 768], color='black', fullscr=True, screen=1, units='norm')     # set the screen and full screen mode
# mywin = visual.Window([640, 360], color='black', fullscr=False, screen=0, units='norm')     # set the screen and full screen mode
# mywin = visual.Window([1920, 1080], color="black", units='norm')    # set the screen and full screen mode

drawTextOnScreen('Loading...')

#Load stimuli to RAM
usefilefname = ['1', '2', '3', '4', '5', '6', 'black']
all_img = []
for im in usefilefname:
    imgPath=image_folder+"/"+str(im)+".png"
    stim = visual.ImageStim( mywin,  image=imgPath )
    stim.size *= .7
    if im == 'black':
        blank = stim
    else:
        all_img.append(stim)

##############
####  Training session
while True:
    isTrianing = True
    drawTextOnScreen('Training session\nPlease wait\nPress space bar to start')
    keys = event.getKeys()
    if 'space' in keys:      # If space has been pushed
        start = time.time()
        drawTextOnScreen('') 

        trial = 1
        for img in stims[str(np.random.randint(1,6))]:
            drawTextOnScreen(f'Trial {trial}/5')
            core.wait(wait)
            clear_output(wait=True)
            timer = random.choice(stim_time)
            drawFixation('trial break', trial_flixation_time-wait)
            drawTrial(img, timer)   # drawTrail(fileName, stimTime, thinkingTime, fixationTime)
            drawFixation('task break', np.random.uniform(task_flixation_time[0], task_flixation_time[1]))
            drawTrial('imagery', timer)

            if trial == 5 :
                break
            trial += 1

        drawTextOnScreen('End of training session')
        core.wait(1)
        drawTextOnScreen('You can ask some question to instructor.\nNo? : space bar')
        _ = event.waitKeys()
        isTrianing = False
        break

################
####### Experiment session
play = True
while play:
    drawTextOnScreen('Experiment session : Press space bar to start')
    keys = event.getKeys()
    if 'space' in keys:      # If space has been pushed
        drawTextOnScreen('Ready')
        core.wait(3)
        start = time.time()
        drawTextOnScreen('') 

        for block in range(num_block) :
            
            if block != 0:
                drawTextOnScreen('RELAX TIME\n*Do not remove your head set\nPress space bar to continue')
                _ = event.waitKeys()
                
            for trial in range(num_trial_per_block):
                for img in stims[str(trial_idx)]:
                    drawTextOnScreen(f'Trial {trial+1}/{num_trial_per_block}')
                    core.wait(wait)
                    clear_output(wait=True)
                    timer = random.choice(stim_time)
                    drawFixation('trial break', trial_flixation_time-wait)
                    drawTrial(img, timer)   # drawTrail(fileName, stimTime, thinkingTime, fixationTime)
                    drawFixation('task break', np.random.uniform(task_flixation_time[0], task_flixation_time[1]))
                    drawTrial('imagery', timer)

#             if block != num_block:  # do not draw fixation on last block
#                 drawTextOnScreen(f'BLOCK REST\n {block_fixation_time} sec.')
#                 core.wait(block_fixation_time)

        drawTextOnScreen('End of experiment, Thank you')
        stop  = time.time()
        print(f"Total experiment time = {(stop-start)/60} ")
        core.wait(10)
        play = False

mywin.close()
core.quit()

Marker string 15,trial break,Fixation
Marker string 15,4,5,2
Marker string 15,task break,Fixation
Marker string 15,4,imagery,2
Total experiment time = 48.668654970328014 
8.1859 	WARNING 	Monitor specification not found. Creating a temporary one...
8.1957 	WARNING 	User requested fullscreen with size [1366  768], but screen is actually [1920, 1080]. Using actual size


SystemExit: 0

/Users/MJ/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
